In [11]:
steps8= """
time : 01-09-2024 10:00 
Steps : 5.0

time : 01-09-2024 11:00 
Steps : 50.0

time : 01-09-2024 12:00 
Steps : 55.0

time : 01-09-2024 13:00 
Steps : 5.0

time : 01-09-2024 14:00 
Steps : 5.0

time : 01-09-2024 15:00 
Steps : 5.0

time : 01-09-2024 16:00 
Steps : 226.0

time : 01-09-2024 17:00 
Steps : 15.0

time : 01-09-2024 18:00 
Steps : 20.0
"""

In [12]:
food8= """
time : 01-09-2024 10:30
foodName : Tea (Protein- 3.58 g, Carbs- 12.64 g, Energy- 101.45 Kcal, Fiber- 0.0 g)

time : 01-09-2024 10:30
foodName : Marie light biscuit (Protein- 0.94 g, Carbs- 7.4 g, Energy- 45.7 Kcal, Fiber- 0.0 g)

time : 01-09-2024 13:30
foodName : White Rice  (Protein- 5.07 g, Carbs- 49.98 g, Energy- 227.63 Kcal, Fiber- 1.79 g)

time : 01-09-2024 13:30
foodName : Drumstick sambar (Protein- 5.18 g, Carbs- 19.7 g, Energy- 140.82 Kcal, Fiber- 4.13 g)

time : 01-09-2024 13:30
foodName : Chow chow poriyal (Protein- 1.29 g, Carbs- 5.03 g, Energy- 29.84 Kcal, Fiber- 2.02 g)

time : 01-09-2024 16:00
foodName : Filter Coffee (Protein- 4.26 g, Carbs- 12.91 g, Energy- 118.06 Kcal, Fiber- 0.0 g)

time : 01-09-2024 18:00
foodName : Masala Vada (Protein- 2.48 g, Carbs- 6.16 g, Energy- 81.41 Kcal, Fiber- 2.09 g)
"""

In [13]:
cgm8="""
CGM readings starting from : 2024-09-01 10:00:00 for every 15 minutes time frame
01-09-2024 10:00 : 121.0 mg/dL
01-09-2024 10:15 : 135.7 mg/dL
01-09-2024 10:30 : 135.7 mg/dL
01-09-2024 10:45 : 169.2 mg/dL
01-09-2024 11:00 : 196.0 mg/dL
01-09-2024 11:15 : 198.9 mg/dL
01-09-2024 11:30 : 187.2 mg/dL
01-09-2024 11:45 : 168.2 mg/dL
01-09-2024 12:00 : 162.6 mg/dL
01-09-2024 12:15 : 154.8 mg/dL
01-09-2024 12:30 : 77.9 mg/dL
01-09-2024 12:45 : 126.1 mg/dL
01-09-2024 13:00 : 126.1 mg/dL
01-09-2024 13:15 : 185.6 mg/dL
01-09-2024 13:30 : 236.7 mg/dL
01-09-2024 13:45 : 251.4 mg/dL
01-09-2024 14:00 : 221.2 mg/dL
01-09-2024 14:15 : 202.2 mg/dL
01-09-2024 14:30 : 172.4 mg/dL
01-09-2024 14:45 : 179.3 mg/dL
01-09-2024 15:00 : 75.0 mg/dL
01-09-2024 15:15 : 95.5 mg/dL
01-09-2024 15:30 : 118.5 mg/dL
01-09-2024 15:45 : 125.8 mg/dL
01-09-2024 16:00 : 125.8 mg/dL
01-09-2024 16:15 : 152.8 mg/dL
01-09-2024 16:30 : 170.7 mg/dL
01-09-2024 16:45 : 178.7 mg/dL
01-09-2024 17:00 : 165.4 mg/dL
01-09-2024 17:15 : 156.8 mg/dL
01-09-2024 17:30 : 150.0 mg/dL
01-09-2024 17:45 : 134.9 mg/dL
01-09-2024 18:00 : 68.5 mg/dL
"""

In [ ]:
from openai import OpenAI
from openai import AzureOpenAI

import os
# from dotenv import load_dotenv
# load_dotenv()

client = AzureOpenAI(
        azure_endpoint="https://oai-tss-sweden.openai.azure.com/",
        api_version="2024-02-01",
        api_key="f0c2aee69c914b35aa55732deb3532be"
        )

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system",
         "content":
         """
              You are a smart glucose monitoring assistant. Analyze continuous glucose monitor (CGM) data recorded every 15 minutes along with corresponding meal intake and activity levels (steps walked) and sleep duration. Review the entire timeline of data, but generate a user-friendly notification only if:
                - Glucose > 180 mg/dL → classify as a "spike" and identify the most probable cause
                - Glucose < 80 mg/dL → classify as a "drop" and identify the most probable cause
                - Any other value (80-180 mg/dL) → classify as "within normal range"

              Input format:

              cgm: Multiline string with CGM readings starting from a specific timestamp in dd-mm-yyyy HH:MM format, with readings every 15 minutes.
              food: Multiline string with food intake information, each entry containing time(dd-mm-yyyy HH:MM), foodName, and nutritional details (Protein, Carbs, Energy, Fiber).
              steps: Multiline string with steps walked and associated timestamps(dd-mm-yyyy HH:MM).
              sleep: single line string with hrs and mins (ex: 5hrs 30mins)

              Evaluate contributing factors when generating glucose alerts. Consider the following:
                - Meals: Check for recent food intake (within 1.5-2 hours of spike), especially high glycemic foods that may cause spikes.
                - Skipped meals or fasting: May lead to drops or instability in glucose.
                - Poor sleep (<6 hours): Especially in the morning (before 10:00 AM), may explain elevated glucose or fatigue.
                - Morning effects (dawn phenomenon): In early hours, natural hormone release (cortisol, growth hormone) may elevate glucose before breakfast.
                - Activity: Low or no post-meal movement may contribute to postprandial spikes due to reduced glucose uptake by muscles.
              Generate a concise alert (max 30 words) that includes the timestamp, glucose value, the most probable reason, and a brief suggestion to improve. Ensure the message is clear and mobile-friendly.

            Example Notification Output:
            1.   Glucose spike (182 mg/dL) at 06:45, likely due to early morning hormone changes. Try avoiding late-night snacks and see how your morning readings go.
            2.   Glucose spike (192 mg/dL), due to white rice at 13:00. Consider a light walk post-meal.
            3.   Glucose drop (72 mg/dL) after skipping lunch. Your energy dipped. Try not to delay meals—have a small snack like a banana or a handful of roasted chana.
            4.   Glucose (105 mg/dL) at 08:15. Slightly elevated, possibly due to stress from poor sleep. A brisk 10-minute walk can help balance energy and improve glucose control.
            5.   Looking great and healthy (92 mg/dL) at 10:00. Balanced meal + solid sleep (7.5h) supports stability.

            **note: make sure the response must be same as one of the example notification and be particular while using the terms spike and drop.**
                
        """
         },
        {
            "role": "user",
            "content": f"""
              cgm:{cgm8},food:{food8},steps:{steps8},sleep:6hrs 30mins
              Based on all the above observations help me suggest the reason for the glucose levels at 01-09-2024 18:00
            """
            
        },
    ],
)

print(completion.choices[0].message.content)

At 18:00 on 01-09-2024, your glucose reading was 68.5 mg/dL, which classifies as a "drop." This likely occurred due to insufficient calorie intake, especially following your post-lunch drop after 13:30. Additionally, low activity levels after meals may have contributed to the instability. To stabilize your glucose, consider having a balanced snack that includes healthy fats and protein.


: 